In [ ]:
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf, params

import numpy as np
import os

from caiman_preprocessing import resave_data_path, get_params_dict
import src.caiman_macros as cm_macros

In [ ]:
# Load and resave the data
# See GitHub Issue #377 - https://github.com/flatironinstitute/CaImAn/issues/377#issuecomment-426740429
# resave_data_path()

In [ ]:
# Get the paths to the data
# Change the list index to select a different file
os.chdir('C:/Users/pugavin/BCB-330-2023-05/pugavin/results')
fnames = ['F147.tif', 'F201.tif']
fname = fnames[0]

In [ ]:
# Retrieve parameters
params_dict = get_params_dict()

# Add the file to the parameter dictionary
params_dict['fnames'] = [fname]

In [ ]:
# Play the movie of the original data
movie_orig = cm.load(params_dict['fnames'])
movie_orig.play()

In [ ]:
# Start a cluster
dview, n_processes = cm_macros.start_cluster()

In [ ]:
# Save the parameter dictionary into a params.CNMFParams object
opts = params.CNMFParams(params_dict=params_dict)

In [ ]:
# Perform rigid motion correction
mc = MotionCorrect(params_dict['fnames'], dview=dview, **opts.get_group('motion'))
mc.motion_correct(save_movie=True)

In [ ]:
# Compare the motion-corrected data with the original data
movie_mc = cm.load(mc.mmap_file)
cm.concatenate([movie_orig - mc.min_mov * mc.nonneg_movie, movie_mc], axis=2).play()

In [ ]:
# Save memory mapped files from motion correction
base_name = params_dict['fnames'][0][:-4] + '_memmap_'
fname_mmap = cm.save_memmap(mc.mmap_file, base_name=base_name, order='C', dview=dview)

In [ ]:
# Load memory mapped files from motion correction
Yr, dims, T = cm.load_memmap(fname_mmap)
images = np.reshape(Yr.T, [T] + list(dims), order='F')

In [ ]:
# Restart the cluster to clean up memory
dview, n_processes = cm_macros.start_cluster(dview)

In [ ]:
# Run CNMF
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
cnm = cnm.fit(images)

In [ ]:
# Plot the contours of any identified components against the correlation image
corr_img = cm.local_correlations(images.transpose((1, 2, 0)))
corr_img[np.isnan(corr_img)] = 0
cnm.estimates.plot_contours_nb(img=corr_img)

In [ ]:
cnm_orig = cnm

In [ ]:
# Rerun CNMF
cnm = cnm.refit(images, dview=dview)

In [ ]:
# Evaluate the quality of inferred spatial components
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

In [ ]:
# Display plot of all components
cnm.estimates.plot_contours_nb(img=corr_img, idx=cnm.estimates.idx_components)

In [ ]:
# Display plot of accepted components
cnm.estimates.nb_view_components(img=corr_img, idx=cnm.estimates.idx_components)

In [ ]:
# Display plot of rejected components, if any
if len(cnm.estimates.idx_components_bad) > 0:
    cnm.estimates.nb_view_components(img=corr_img, idx=cnm.estimates.idx_components_bad)

In [ ]:
# Keep only the accepted components
cnm.estimates.select_components(use_object=True)

In [ ]:
# View a plot of the results
cnm.estimates.nb_view_components(img=corr_img, denoised_color='red')

In [ ]:
# View a movie of the results
cnm.estimates.play_movie(images)

In [ ]:
# Save results
cnm.save(cnm.mmap_file[:-4] + 'hdf5')

In [ ]:
# Stop the cluster
cm.stop_server(dview=dview)